In [7]:
import pandas as pd, numpy as np, json

In [124]:
base_url='http://www.cdep.ro'

In [125]:
nmembers=json.loads(open('data/nmembers.json','r').read())

In [126]:
img=json.loads(open('data/img.json','r').read())

In [140]:
records=json.loads(open('data/records.json','r').read())

In [141]:
records[90]

{'votid': '318',
 'vot': 'DA',
 'grup': 'neafiliat',
 'name': 'Lificiu Petru',
 'camera': '2'}

In [151]:
nmembers['Calimente Mihaita']

{'Name': {'full': 'Mihăiţă CALIMENTE', 'simple': 'Calimente Mihaita'},
 'Photo': ['http://www.cdep.ro/parlamentari/l1996/caliment.jpg',
  'http://www.cdep.ro/parlamentari/l2008/Calimente_Mihaita.jpg',
  'http://www.cdep.ro/parlamentari/l2012/Calimente_Mihaita.jpg',
  'http://www.cdep.ro/parlamentari/l2004/CalimenteMihaita.jpg'],
 'UserID': ['5', '58', '50', '60'],
 'Camera': ['DEPUTAT', 'DEPUTAT', 'DEPUTAT', 'DEPUTAT'],
 'Link': ['http://www.cdep.ro/pls/parlam/structura2015.mp?idm=5&leg=1996&cam=2',
  'http://www.cdep.ro/pls/parlam/structura2015.mp?idm=58&leg=2008&cam=2',
  'http://www.cdep.ro/pls/parlam/structura2015.mp?idm=50&leg=2012&cam=2',
  'http://www.cdep.ro/pls/parlam/structura2015.mp?idm=60&leg=2004&cam=2'],
 'Parties': [{'party': 'PNTCD',
   'start': '1996-11-27',
   'end': '2000-02-01',
   'judet': 'ARAD'},
  {'party': 'Independent',
   'start': '2000-02-01',
   'end': '2000-11-30',
   'judet': 'ARAD'},
  {'party': 'PNL',
   'start': '2008-12-19',
   'end': '2012-12-20',
  

In [130]:
{i['vot'] for i in records}

{'-', 'AB', 'DA', 'NU'}

In [131]:
votes=json.loads(open('data/votes.json','r').read())

In [132]:
votes[9]

[{'votid': '313',
  'descriere': 'Proiectul de Hotărâre privind atribuirea unor spaţii din Palatul Parlamentului în folosinţa gratuită a Clubului Parlamentarilor Români',
  'time': '06.02.2006 16:47',
  'camera': '2'}]

In [133]:
def get_allegiance(allegiance,voteid,outcome,names):
    if voteid not in allegiance:
        allegiance[voteid]={}
    if outcome not in allegiance[voteid]:
        allegiance[voteid][outcome]=[]
    for name in names:
        allegiance[voteid][outcome].append(name)
    return allegiance

In [134]:
votes_dict={v[0]['votid']:v[0] for v in votes}
#votes_dict={v['votid']:v for v in votes}

In [135]:
votes_dict['388']

{'votid': '388',
 'descriere': 'Pl-x 585/2005',
 'time': '14.02.2006 12:50',
 'camera': '2'}

In [136]:
def senate(camera):
    if camera=='1':return 'Senatori'
    else: return 'Deputați'

In [154]:
import unidecode

In [155]:
def name_normalizer(name):
    return unidecode.unidecode(name)

In [165]:
len(records)

3382156

In [168]:
eu_allegiance={}
eu_vt={}
eu_joint_allegiance={}
eu_joint_vt={}
records_parsed=set()

In [169]:
for i in range(len(records[:])):
    if (i%10000==0):
        print(i)
    if i not in records_parsed:
        j=records[i]
        vote=votes_dict[j['votid']]
        ts=pd.to_datetime('-'.join(vote['time'].split(' ')[0].split('.')[::-1]))
        year=int(vote['time'].split(' ')[0][-4:])
        camera=senate(vote['camera'])
        if camera not in eu_vt:eu_vt[camera]={}
        if year not in eu_vt[camera]:eu_vt[camera][year]=[]
        if camera not in eu_allegiance:eu_allegiance[camera]={}
        if year not in eu_allegiance[camera]:eu_allegiance[camera][year]={'Name':{},'Group':{},'Party':{},'Country':{}}  
        if year not in eu_joint_vt:eu_joint_vt[year]=[]
        if year not in eu_joint_allegiance:eu_joint_allegiance[year]={'Name':{},'Group':{},'Party':{},'Country':{}}    
        if vote['descriere'] not in ["Modification de l'ordre du jour"]:
                #name=' '.join(j['name'].split(' ')[::-1])
                name=name_normalizer(j['name'])
                if name not in nmembers:
                    print(name)
                else:
                    member=nmembers[name]
                    dummy={}
                    outcome=j['vot']
                    dummy['Vote']=j['votid']
                    dummy['Name']=[member['Name']['full']]
                    dummy['Outcome']=outcome
                    dummy['Year']=year
                    dummy['Group']=[]
                    dummy['Party']=[]
                    dummy['Country']=[]
                    for k in member['Parties']:
                        if pd.to_datetime(k['start'])<ts<pd.to_datetime(k['end']):
                            dummy['Party'].append(k['party'])
                    for k in member['Groups']:
                        if pd.to_datetime(k['start'])<ts<pd.to_datetime(k['end']):
                            dummy['Group'].append(k['group'])
                    for k in member['Countries']:
                        if pd.to_datetime(k['start'])<ts<pd.to_datetime(k['end']):
                            dummy['Country'].append(k['country'])
                    eu_vt[camera][year].append(dummy)
                    eu_joint_vt[year].append(dummy)
                    for allegiance_type in eu_allegiance[camera][year].keys():
                        eu_allegiance[camera][year][allegiance_type]=\
                            get_allegiance(eu_allegiance[camera][year][allegiance_type],j['votid'],
                                       outcome,dummy[allegiance_type])
                        eu_joint_allegiance[year][allegiance_type]=\
                            get_allegiance(eu_joint_allegiance[year][allegiance_type],j['votid'],
                                       outcome,dummy[allegiance_type])
        records_parsed.add(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [170]:
eu_allegiance['Joint']=eu_joint_allegiance
eu_vt['Joint']=eu_joint_vt

In [171]:
open('data/eu_allegiance.json','w').write(json.dumps(eu_allegiance))

1253594984

In [172]:
open('data/eu_vt.json','w').write(json.dumps(eu_vt))

1890558212